In [1]:
import configparser
import Ipynb_importer
from graph import Graph
from config import Config
from SDNE import SDNE
from utils_ import *
import scipy.io as sio
import time
import copy
import torch.optim as optim
from optparse import OptionParser
import os

importing Jupyter notebook from graph.ipynb
importing Jupyter notebook from utils_.ipynb
importing Jupyter notebook from config.ipynb
importing Jupyter notebook from SDNE.ipynb
importing Jupyter notebook from RBM.ipynb


In [2]:
config=Config('ca-grqc.ini')
train_graph_data = Graph(config.train_graph_file, config.ng_sample_ratio)
   
if config.origin_graph_file:
    origin_graph_data = Graph(config.origin_graph_file, config.ng_sample_ratio)

if config.label_file:
    #load label for classification
    train_graph_data.load_label_data(config.label_file)

negative Sampling


graph.ipynb:53: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  "                self.adj_matrix = sio.loadmat(file_path)[\"traingraph_sparse\"].tocsr()\n",
graph.ipynb:54: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  "            self.N, _ = self.adj_matrix.get_shape()\n",


negative Sampling done
Vertexes : 5242  Edges : 14496 ngSampleRatio: 0.500000
negative Sampling
negative Sampling done
Vertexes : 5242  Edges : 14496 ngSampleRatio: 0.500000


In [3]:
config.struct[0] = train_graph_data.N
model = SDNE(config)
model.do_variables_init([train_graph_data])
embedding = None

rbm init complete
rbm epochs: 0 error :  tensor(14719575., device='cuda:0', dtype=torch.float64)


In [4]:
mini_batch = train_graph_data.sample(config.batch_size, do_shuffle = False)

In [5]:
mini_batch

{'X': array([[0., 1., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'adjacent_matriX': array([[0., 1., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])}

In [33]:
optimizer=optim.SGD(model.parameters(),lr=0.01,weight_decay=0.01)

In [38]:
model.cuda()
import torch
loss=model(torch.tensor(mini_batch.X,device='cuda'),
      torch.tensor(mini_batch.adjacent_matriX,device='cuda'),config)
loss

/home/zda/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor(1461218.6183, device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [ ]:
for i in range(1000000):
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss=model(torch.tensor(mini_batch.X,device='cuda'),
          torch.tensor(mini_batch.adjacent_matriX,device='cuda'),config)
    if i%50000==0:
        print(loss)

tensor(1989731.5248, device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor(1525971.5023, device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
tensor(1882137.8118, device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)


In [8]:
for x in model.parameters():
    print(x.name,x.data)

None tensor([[-0.0659, -0.0575,  0.0107,  ..., -0.0483,  0.0206, -0.0023],
        [-0.0944,  0.0179, -0.0209,  ..., -0.1110, -0.0596,  0.0950],
        [ 0.0192,  0.0367, -0.0385,  ...,  0.0208,  0.0320,  0.0209],
        ...,
        [-0.0174, -0.0555, -0.0832,  ...,  0.0549,  0.1306, -0.0657],
        [-0.0928, -0.0108, -0.0028,  ..., -0.0247,  0.0014,  0.0010],
        [-0.0496, -0.0130, -0.0168,  ..., -0.0281, -0.0570, -0.0442]],
       device='cuda:0', dtype=torch.float64)
None tensor([0.0380, 0.0401, 0.0372, 0.0379, 0.0369, 0.0372, 0.0372, 0.0388, 0.0380,
        0.0382, 0.0359, 0.0393, 0.0387, 0.0397, 0.0373, 0.0386, 0.0372, 0.0346,
        0.0409, 0.0392, 0.0401, 0.0408, 0.0386, 0.0351, 0.0385, 0.0384, 0.0385,
        0.0401, 0.0378, 0.0374, 0.0408, 0.0382, 0.0348, 0.0401, 0.0403, 0.0376,
        0.0397, 0.0372, 0.0417, 0.0375, 0.0389, 0.0365, 0.0410, 0.0374, 0.0403,
        0.0389, 0.0385, 0.0368, 0.0397, 0.0356, 0.0387, 0.0394, 0.0350, 0.0388,
        0.0390, 0.0389, 0.0404,

In [9]:
while (True):
        mini_batch = train_graph_data.sample(config.batch_size, do_shuffle = False)
        if embedding is None:
            embedding = model.get_embedding(mini_batch)
        else:
            embedding = np.vstack((embedding, model.get_embedding(mini_batch))) 
        if train_graph_data.is_epoch_end:
            break

AttributeError: 'SDNE' object has no attribute 'get_embedding'